# Explore warp file generated by ants

The ants software, can perform registration of a moving image versus a fixed or reference image. When the transformation is non-linear, ants output a *warp* file and an *inverse-warp* file.
# The warp file
* It is a vectorial field that encode voxel per voxel the trasformation to register the image from moving-space to fixed space. It assumes they are already in the same space; for example thanks to an affine registration, previoulsy performed. 

* It is defined in reference, or fixed image space . This means there is a direct correspondance between the dimensions of the warp and the one of the fixed image. It has three channels. So there are three values for each coordinate of the fixed space.
Eg: If a fixed image has dimension nxmxh the Warp image dimension is nxmxhx3

* The Warp image information is encoded in two entities:
   * **the coordinates of the warp-image voxel**. These should be interpreted as a "destination point" of the trasformation from the moving space to the fixed space. Now you should wonder: "these are the destination point, but where are encoded the point I have to move versus the destination points?" The answer will come very soon.
   
   * the **voxels values** or **vectors**. Each coordinate of the warp-image, corresponding to a coordinate in the fixed space, has three voxels associated. These three values are 3-D vectors that point somewhere in the moving space. They encode the displacement in mm of the moving point I want to move, with respect to the position of the coordinate where they are stored. So this is the answer to the previous question: when ants compute the registration these vectors indicate which points in moving space will be moved in fixed space, versus the coordinate where they are stored [1,2]. 

# The inverse warp file
* The inverse warp file is defined similarly to the warp. The difference is that it is a vectorial field encoding the "iverse" trasformation from fixed-space to moving space, voxel per voxel. It also assume the moving and fixed space were already normalized with a linear registration.

* It is defined in moving space. So the dimension of the nifti image are the one of the moving image. Similarly to the warp file has three channels, or values for each coordinate of the moving space.

* As for the Warp image the information is encoded in the two entities:
   * **the coordinates of the inverse-warp-image voxel**. These should again be interpreted as a "destination point" of the trasformation, that is now from reference or fixed space to moving space. 
   
   * the **voxels values** or **vectors**. Each coordinate of the inverse-warp-image, corresponding to a coordinate in the fixed space, has as before 3 voxels associated, that are 3-D vectors. These vectors point somewhere in the fixed space. As before they encode the displacement in mm of the point in the fixed space, with respect to the position of the coordinate where they are stored.  When ants compute the registration these vectors indicate which points in fixed space will be moved in moving space, versus the coordinate where they are stored. <br>
   
<br>   
**NB The inverse trasformation in inverse-warp is not the exact inverse of the warp transformation** 
 
So to study the transformation per-se is advisable to use the warp-field that encode the original transformation from moving to fixed, that is directly optimized by ants [3]. 
 
### Source: 
[1] https://sourceforge.net/p/advants/discussion/840261/thread/b6fee8f3a0/ <br>
[2] https://sourceforge.net/p/advants/discussion/840261/thread/915b1b22/  <br>
[3] https://raw.githubusercontent.com/stnava/ANTsDoc/master/ants2.pdf

# Some Experiment to test warp definition

To test what ants sofware really do during the transformation:
* Select a point in the warp that corresponds to quite high vectors values/displacements. These coordinates will be the destination coordinates dest_c of the transformation considered.
* Then, let's define a image with a checkpoint. This image has the same dimensions as the moving image. It is zero everywhere, but for one point P (xm,ym,zm) that has value 100. This point is the one I expect to be moved in the destination coordinates dest_c. 
It is compute as the difference (or the sum.. see below) of the coordinates dest_c and vector values ecoded by  point dest_c in the warp image.

* Then I apply the transformation encoded in the warp to this checkpoint image, using ants. 
* I laod the warped image resulted from previous step. I check where it has voxels values >0. These will be the position where the check-point was moved in the reference space, by the transformation. Let's call these point P_warped
* NB: since I am using linear interpolation it is expected that the point is spread in more than one point in the warped image. So P_warped will be probably a set of points.

### TEST 1:
Check if  the coordinates P_warped (where the point  was moved in warped image) correspond to dest_c. The dest_c were the destination coordinate in the warp considered from the begining
### TEST 2: 
Consider the original point coordinate in the moving space P=(xm,ym,zm) of the the point. Consider P_warped, obtained after registration. The diffierence between these coordinates is the one encoded in the InverseWarp at position P=(xm,ym,zm)?

I managed to find positive answer for both tests but seems that all not dimensions of the vectors have the same orientations (it is necessary to multiply by -1 some vector-value). More details are reported in the code. The results are consistent for multiple points dest_c considered in the warp.

In [1]:
import nibabel as nib
import os
import numpy as np

Let's define the paths..

In [2]:
dir_data="/home/chiara/Experiments/Tract_adapt/explore_warp_format"

#path of the warp and of the inverse warp
warp_path = f"{dir_data}/sub-1278__affined_T1_SyN1Warp.nii.gz"
warp_inv_path=f"{dir_data}/sub-1278__affined_T1_SyN1InverseWarp.nii.gz"

#image moving path
img_moving_path=f"{dir_data}/sub-1278__affined_T1.nii.gz"

#image fixed-reference path
img_fixed_path=f"{dir_data}/IFOF_L_MultiLabel_affined_MNI.nii.gz"

#path to images I will create to test the warping
#this images will with the same dimension as the moving images, and they wuill be xero everywhere 
#but for one point
img_point_path=f"{dir_data}/moving_space_point.nii.gz"
img_point_warped=f"{dir_data}/ref_space_point_warped.nii.gz"

Load the nifti image of the warp, the inverse warp and of the image in the original moving space.

In [3]:
#load images

#warp image
warp = nib.load(warp_path)
warp_np=warp.get_fdata()

#inverse warp image
inv_warp = nib.load(warp_inv_path)
inv_warp_np= inv_warp.get_fdata()

#moving image, that was already linearly registered with reference image
img_mov = nib.load(img_moving_path)
img_mov_np = img_mov.get_fdata()


Then, select a point in the warp that corresponds to quite high vectors values/displacements. Store the coordinates and the value of the vector. These coordinates will be the destination coordinates

In [4]:
#destination_coordinate - or coordinate of the warp tested
#these are all coordinates with quite big displacment

#dest_c=np.array([53, 120, 56])
#dest_c=np.array([60, 125, 60])
#dest_c=np.array([80, 125, 80])
dest_c=np.array([80, 125, 90])


#the destination coordinate contains number that indicate the realtive position of 
#coordinate in the moving space that will be warped to destination_coordinates
w= np.array(warp_np[dest_c[0], dest_c[1],dest_c[2],0])

#%%
print("Coordinate of the warp considered (the destination coordinate)")
print(dest_c)
print("Vectors values, indicating the position of the point in the moving space that will be warped versus destinantion coordinate in fixed space")
print(w)

Coordinate of the warp considered (the destination coordinate)
[ 80 125  90]
Vectors values, indicating the position of the point in the moving space that will be warped versus destinantion coordinate in fixed space
[ 7.3114028   8.56540203 -9.1455574 ]


Create the checkpoint image in moving space.

In [5]:
#I define a image with a checkpoint. This image has the same dim as the moving one but is all zero,
#but for the point I expect to be moved in the destination coordinates by the 
#warp encodaed at point dest_c.
#This is the point pointed by the  vector w  in coordinate dest_c of the warp image

img_tmp_point = np.zeros(img_mov_np.shape)

#coordinates in the moving image that will be warped to destination coordinate

xm = dest_c[0] - round(w[0])
ym = dest_c[1] - round(w[1])
zm = dest_c[2] + round(w[2])

print("Point in the moving spaced drawn at: ")
#coordinates I thought were displaced to dest_c during warp
print(xm, ym, zm)
img_tmp_point[xm, ym, zm ] = 100
              
print("\nSave the check-point point image to files")  
out_nii=nib.Nifti1Image(img_tmp_point, affine=img_mov.affine, header=img_mov.header)
out_nii.to_filename(img_point_path)


Point in the moving spaced drawn at: 
73 116 81

Save the check-point point image to files


In [6]:
print("Apply the registration on the image with the checkpoint\n")

bash_c=f"antsApplyTransforms -d 3 -i {img_point_path} -r {img_fixed_path} -t {warp_path} -n linear -o {img_point_warped}" 
os.system(bash_c)

Apply the registration on the image with the checkpoint



0

Let's check if everything is ok...

In [7]:
print(f"The inverse warp (defined in the moving space) at the coordinate {xm, ym, zm} ")
print(inv_warp_np[xm,ym,zm])
print()

#%%
#I check how much the point moved

img_p_warped = nib.load(img_point_warped)
img_p_warped_np = img_p_warped.get_fdata()

ijk_point_warped = (np.array(np.where(img_p_warped_np>0)).T)
print("\n\n------------------------------------------------------")
print("Do the warped coordinate in fixed space correspond to the coordiante where I read the warp?  ")
print("Warped coordinate:")
print(ijk_point_warped)
print()
print("Coordinate in the warp in the ref space considered")
print(dest_c)
print("Yesss  :) \n  But the third dimension of the warp has to be multiplied by -1 to be coherent with the other dimensions")
#%%

print("\n\n-----------------------------")
print(f"Check: does the difference between:")
print(f"- coords of point in moving space:\n { xm, ym, zm }")
print(f"- and corresponding points in warped space\n {ijk_point_warped}")
print(f" ---> correspond to the value indicated by the inverse warp in the coordiante in the moving space:")
print(f"{inv_warp_np[xm,ym,zm]} ?")

print(f"\n Yes :))")
print("NB: the intepolation of the registration is linear so it is expected a single point in moving space is spread in the fixed space in multiplt voxels, after registration")
print(".. But strangely I have to multiply by -1 the last coordinate of the inv warp. This behaviour is shared among all coordinates points I checked")



The inverse warp (defined in the moving space) at the coordinate (73, 116, 81) 
[[-7.25318146 -8.36985779  9.06699276]]



------------------------------------------------------
Do the warped coordinate in fixed space correspond to the coordiante where I read the warp?  
Warped coordinate:
[[ 80 124  90]
 [ 80 124  91]
 [ 80 125  90]
 [ 80 125  91]
 [ 81 123  90]
 [ 81 124  89]
 [ 81 124  90]]

Coordinate in the warp in the ref space considered
[ 80 125  90]
Yesss  :) 
  But the third dimension of the warp has to be multiplied by -1 to be coherent with the other dimensions


-----------------------------
Check: does the difference between:
- coords of point in moving space:
 (73, 116, 81)
- and corresponding points in warped space
 [[ 80 124  90]
 [ 80 124  91]
 [ 80 125  90]
 [ 80 125  91]
 [ 81 123  90]
 [ 81 124  89]
 [ 81 124  90]]
 ---> correspond to the value indicated by the inverse warp in the coordiante in the moving space:
[[-7.25318146 -8.36985779  9.06699276]] ?

 Yes :))
N